In [ ]:
import yfinance as yf
import pandas as pd
import logging
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

# Disclaimer: yfinance is outdated. 
# The ".info" and ".get_info" functions no longer work.
# When trying to run yfinance, 
# you will get this debug:

# DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): query2.finance.yahoo.com:443
# DEBUG:urllib3.connectionpool:https://query2.finance.yahoo.com:443 "GET /v8/finance/chart/TSLA?period1=1262322000&period2
# And if you attempt to visit "https://query2.finance.yahoo.com/" you will get an error from Yahoo.
# No data exists at this HTTPS connection. So it fails to work.
# I tried to resolve this by getting historical data (before I knew the problem)
# It does not work, the query is completely nonfunctional.

# "raise JSONDecodeError("Expecting value", s, err.value) from None"
# "json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)"
# There is no data to grab.

# apple = yf.Ticker("AAPL") This was a test line
#logging.basicConfig(level=logging.DEBUG)

# try:
    # tesla_stock = yf.download("TSLA", start="2010-01-01")
    # gamestop_stock = yf.download("GME", start="2010-01-01")
# except Exception as e:
    # print("Error downloading")

# print(tesla_stock.head())
# print(gamestop_stock.head())

# Below code is using the newest yfinance version which is not the one in the assignment details


tesla = yf.Ticker("TSLA")
gme = yf.Ticker("GME")


# stock_data, title as placeholders for later specification
def make_graph(stock_data, title):
    plt.figure(figsize=(10, 5))
    plt.plot(stock_data["Date"], stock_data["Close"])
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Stock Price")
    plt.grid(True)
    


tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print(tesla_data.head())

gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
print(gme_data.head())

def scrape_revenue(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.find_all("table")
    df = pd.read_html(str(tables[1]))[0]
    df.columns = ["Date", "Revenue"]
    df["Revenue"] = df["Revenue"].str.replace(r"[$,]", "", regex=True)
    df.dropna(inplace=True)
    return df
tesla_url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
gme_url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

tesla_revenue = scrape_revenue(tesla_url)
tesla_revenue.columns = ['Date','Revenue']
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',|\$',"", regex=True)

gme_revenue = scrape_revenue(gme_url)
gme_revenue.columns = ['Date','Revenue']
gme_revenue["Revenue"] = gme_revenue['Revenue'].str.replace(',|\$',"", regex=True)


print(tesla_revenue.tail())
print(gme_revenue.tail())

make_graph(tesla_data, "Tesla Stock Price Over Time")
make_graph(gme_data, "GameStop Stock Price Over Time")
plt.show()